In [1]:
from glob import glob #retriving an array of files in directories
from tqdm import tqdm
import os
import csv
import random
import pydicom
import numpy as np
import pandas as pd
from skimage import io
from skimage import measure
from skimage.transform import resize
import cv2
import tensorflow as tf
from tensorflow import keras

from matplotlib import pyplot as plt
import matplotlib.patches as patches
import shutil
import pydicom as dicom
from tqdm import tqdm

In [14]:
def make_dirs(out):
    os.makedirs(out)
    os.makedirs(out+"train/")
    os.makedirs(out+"test/")
    os.makedirs(out+"train/NORMAL/")
    os.makedirs(out+"train/PNEUMONIA/")
    os.makedirs(out+"test/NORMAL/")
    os.makedirs(out+"test/PNEUMONIA/")

In [26]:
output_dir = "./dataset/Pneumonia/rsna_data/"
sub = pd.read_csv("./input/kaggle_rsna/stage_2_train_labels.csv")
detail = pd.read_csv("./input/kaggle_rsna/stage_2_detailed_class_info.csv")
detail.tail()

,patientId,class
30222,c1ec14ff-f6d7-4b38-b0cb-fe07041cbdc8,Lung Opacity
30223,c1edf42b-5958-47ff-a1e7-4f23d99583ba,Normal
30224,c1f6b555-2eb1-4231-98f6-50a963976431,Normal
30225,c1f7889a-9ea9-4acb-b64c-b737c929599a,Lung Opacity
30226,c1f7889a-9ea9-4acb-b64c-b737c929599a,Lung Opacity


In [5]:
#converting dcm to jpg
folder_path = "./input/kaggle_rsna/stage_2_train_images/"
images_path = os.listdir(folder_path)
os.makedirs("./input/kaggle_rsna/stage_2_train_images2/")
jpg_folder_path = "./input/kaggle_rsna/stage_2_train_images2/"
for n, image in enumerate(images_path):
        ds = dicom.dcmread(os.path.join(folder_path, image))
        rows = []
        pixel_array_numpy = ds.pixel_array
        image = image.replace('.dcm', '.jpg')
        cv2.imwrite(os.path.join(jpg_folder_path, image), pixel_array_numpy)

In [6]:
dis_type = detail["class"]
dis_name = detail["patientId"]
name = sub["patientId"]
value = sub["Target"]
name = [name[i]+".jpg" for i in range(len(name))]
dis_name = [dis_name[i]+".jpg" for i in range(len(dis_name))]
img = glob("./input/kaggle_rsna/stage_2_train_images2/*") #Getting all images in this folder
filenames = os.listdir("./input/kaggle_rsna/stage_2_train_images2/")

In [7]:
#list.index(element)
normal,pneumonia = [],[]
normal_name,pneumonia_name = [],[]
for i in tqdm(range(len(filenames))):
    ind = name.index(filenames[i])
    ind1 = dis_name.index(filenames[i])
    if(value[ind] == 0 and dis_type[ind] == "Normal"):
        normal.append(img[i])
        normal_name.append(filenames[i])
    elif(value[ind] == 1):
        pneumonia.append(img[i])
        pneumonia_name.append(filenames[i])

100%|██████████| 26684/26684 [00:11<00:00, 2242.14it/s]


In [21]:
random.shuffle(normal)
random.shuffle(pneumonia)
normal_train,normal_test = normal[:int(len(normal)*.9)],normal[int(len(normal)*.9):]
pneumonia_train,pneumonia_test = pneumonia[:int(len(pneumonia)*.9)],pneumonia[int(len(pneumonia)*.9):]

In [25]:
len(pneumonia_test)

602

In [27]:
len(normal_test)
try:
    make_dirs(output_dir)
except:
    pass

In [28]:
for image in normal_train:
    shutil.copy(image,output_dir+"train/NORMAL")
for image in normal_test:
    shutil.copy(image,output_dir+"test/NORMAL")  
for image in pneumonia_train:
    shutil.copy(image,output_dir+"train/PNEUMONIA")
for image in pneumonia_test:
    shutil.copy(image,output_dir+"test/PNEUMONIA")

The above code is for processing rsna data and storing it in the desired format.